This notebook contains code for implementing the Heston model and calibrating model parameters, followed by Monte Carlo simulations to validate the model.

# Importing Libraries

In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import plotly.graph_objects as go

from scipy.integrate import quad
from scipy.optimize import minimize
from datetime import datetime as dt

from nelson_siegel_svensson.calibrate import calibrate_nss_ols

# Implementing Heston Model
Heston model known parameters (from market data):
- initial asset price (S0) 
- risk-free interest rate (r) 
- time to maturity(T) 
- strike price (K)

unknown parameters (determined through optimization algorithms and calibration techniques):
- initial volatility (v0),
- speed of mean-reversion (kappa)
- long term mean of volatility (theta)
- volatility of volatility (sigma)
- correlation between the two wiener processes for asset price and volatility (rho)

Heston equations (SDEs, PDE, characteristic function) are from this paper:
https://www.maths.univ-evry.fr/pages_perso/crepey/Finance/051111_mikh%20heston.pdf

In [51]:
# Defining heston model characteristic function
# Characteristic function derived by assuming form and substituting into Heston PDE (from )

def heston_char_func(phi, S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd):

    # commonly used term
    rspi = rho*sigma*phi*1j

    # constants
    a = kappa*theta
    b = kappa+lambd

    # d and g parameter in heston characteristic function
    d = np.sqrt((rspi - b)**2 + sigma**2 * (phi*1j+phi**2))
    g = (b-rspi+d) / (b-rspi-d)

    exp1 = np.exp(r*phi*1j*tau)
    term1 = S0**(1j*phi) * ((1-g*np.exp(d*tau))/(1-g))**(-2*a/sigma**2)
    exp2 = np.exp(a*tau/sigma**2 * (b-rspi+d) + v0/sigma**2 * (b-rspi+d)*((1-np.exp(d*tau))/(1-g*np.exp(d*tau))))

    return exp1*term1*exp2

In [20]:
# Using numerical integration to simplify the process of finding price
def heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd):
    args = (S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd)
    
    # 10000 steps and range of 1-100, each step (dphi) is 0.01
    P, umax, N = 0, 100, 10000
    dphi=umax/N 
    
    # Loop through all the steps and summing the value of integral at each step
    for i in range (1,N):
        phi = dphi * (2*i + 1)/2
        
        # P is 0 intially, and the value of the integral * dphi is added to P each step 
        P += ((np.exp(r*tau)*heston_char_func(phi-1j,*args) - K * heston_char_func(phi,*args)) / (1j*phi*K**(1j*phi))) * dphi

    # Substituting the value of the integral into equation for cost and taking the real value
    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

In [21]:
S0 = 105
K = 110
v0 = 0.04
tau = 0.1
r = 0.05
sigma = 0.6
rho = -0.8
kappa = 3
theta = 0.04
lambd = 0.8

heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd)

np.float64(0.5687456253728729)

# Getting Real World Data
## Interest rate

In [61]:
# Interest rate data for 2nd October 2023 from US Department of the Treasury
rates_df = pd.read_csv('/Users/wongmarco/Downloads/interest_rates.csv', index_col= 0)
rates_df['Date'] = pd.to_datetime(rates_df['Date'], format='%m/%d/%Y')
rates_df.set_index('Date', inplace = True)

# Change all data from percentage to float
rates_df = rates_df.astype(float)/100

In [62]:
# Using Nelson Siegel Svennson model (parametric) to analyse yield curve using ordinary least squares
maturities = rates_df.columns.astype(float).to_numpy()
rates_df['curve_fit'] = rates_df.apply(lambda row: calibrate_nss_ols(maturities, row.values)[0], axis=1)

## Option data
Option data is downloaded from optionsdx in form of CSV files (This project doesn't require real time quoting of option data)

In [82]:
# File below contains EOD option data for NVDA in October 2023
options_df = pd.read_csv('/Users/wongmarco/Downloads/nvda_eod_2023q4-jf5cdq/nvda_eod_202310.txt')

Data Cleaning and Transformation

In [84]:
pd.set_option('display.max_columns', 33)
options_df.columns = options_df.columns.str.replace('[\[\] ]', '', regex=True)
options_df['QUOTE_DATE'] = pd.to_datetime(options_df['QUOTE_DATE'])
options_df['EXPIRE_DATE'] = pd.to_datetime(options_df['EXPIRE_DATE'])
options_df = options_df.replace(' ',np.nan )
options_df['C_VOLUME'] = options_df['C_VOLUME'].astype(float)
options_df.dropna()
options_df = options_df[options_df['DTE'] != 0]


In [58]:
# Removing inaccurate data where option last trading price = 0 and trade volume is null 
options_df = options_df[(options_df['C_LAST'] != 0) & (options_df['P_LAST'] != 0)]

In [59]:
options_df

# Columns in dataframe: 
    # Time: (time in unix, quote time, quote date, quote hour of time, expire date, expire time in unix, day to expiration)
    # Greeks: Delta (Call and Put), Gamma, Vega, Theta, Rho
    # Other data (Call and Put):  implied volatility, trading volume, last traded price, size (open interest)
    # Strike: Strike price, strike distance (absolute/ percentage distance) between stock and strike price

,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,P_BID,P_ASK,P_SIZE,P_LAST,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT
0,1696276800,2023-10-02 16:00,2023-10-02,16.0,447.79,2023-10-06,1696622400,4.00,1.00000,0.00000,0.00000,-0.01072,0.00863,NaN,1.0,373.60,6 x 7,377.35,377.75,70.0,0.00,0.01,0 x 149,0.01,0.00000,0.00000,-0.00020,-0.00404,-0.00023,4.645560,0.000000,377.8,0.844
1,1696276800,2023-10-02 16:00,2023-10-02,16.0,447.79,2023-10-06,1696622400,4.00,1.00000,0.00000,0.00000,-0.01176,0.01008,NaN,0.0,390.50,6 x 7,367.35,367.75,80.0,0.00,0.01,0 x 122,0.02,-0.00018,0.00000,0.00036,-0.00481,0.00000,4.312810,0.000000,367.8,0.821
3,1696276800,2023-10-02 16:00,2023-10-02,16.0,447.79,2023-10-06,1696622400,4.00,1.00000,0.00000,0.00000,-0.01503,0.01317,NaN,0.0,338.45,11 x 10,346.75,348.20,100.0,0.00,0.01,0 x 62,0.01,-0.00047,0.00000,0.00010,-0.00408,-0.00024,3.759380,0.000000,347.8,0.777
8,1696276800,2023-10-02 16:00,2023-10-02,16.0,447.79,2023-10-06,1696622400,4.00,1.00000,0.00000,0.00000,-0.02221,0.01919,NaN,0.0,280.95,7 x 7,297.10,298.20,150.0,0.00,0.01,0 x 35,0.01,0.00000,0.00000,0.00031,-0.00461,0.00000,2.761420,0.000000,297.8,0.665
12,1696276800,2023-10-02 16:00,2023-10-02,16.0,447.79,2023-10-06,1696622400,4.00,1.00000,0.00000,0.00000,-0.02673,0.02282,NaN,0.0,245.30,16 x 1,266.75,268.30,180.0,0.00,0.01,0 x 39,0.01,-0.00054,-0.00002,-0.00004,-0.00465,-0.00014,2.314950,0.000000,267.8,0.598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58501,1698782400,2023-10-31 16:00,2023-10-31,16.0,407.80,2026-01-16,1768597200,808.04,0.37814,0.00137,2.28159,-0.07744,2.25382,0.459900,10.0,52.10,106 x 63,52.05,52.90,700.0,297.30,304.40,39 x 39,273.01,-0.75315,0.00209,1.48895,-0.02030,-3.29247,0.453180,0.000000,292.2,0.717
58511,1698782400,2023-10-31 16:00,2023-10-31,16.0,407.80,2026-01-16,1768597200,808.04,0.30340,0.00128,2.10432,-0.06980,1.87769,0.455910,1.0,38.55,132 x 159,38.75,39.70,800.0,389.25,396.65,58 x 23,372.27,-0.82882,0.00199,0.97856,-0.01447,-2.19033,0.476280,0.000000,392.2,0.962
58512,1698782400,2023-10-31 16:00,2023-10-31,16.0,407.80,2026-01-16,1768597200,808.04,0.29677,0.00122,2.08308,-0.06909,1.84229,0.455480,0.0,43.68,134 x 68,37.70,38.45,810.0,398.55,404.20,45 x 3,348.95,-1.00000,0.00000,0.00000,0.00000,0.00000,NaN,0.000000,402.2,0.986
58518,1698782400,2023-10-31 16:00,2023-10-31,16.0,407.80,2026-01-16,1768597200,808.04,0.25484,0.00116,1.93415,-0.06305,1.61098,0.452830,6.0,31.40,151 x 1,31.00,31.50,880.0,467.00,477.00,2 x 15,428.81,-1.00000,0.00000,0.00000,0.00000,0.00000,NaN,0.000000,472.2,1.158


In [126]:
# Data processing

# Getting a dataframe with only the parameters needed for the actively traded call options
traded_options = options_df[(options_df['C_VOLUME'] > 500)][['UNDERLYING_LAST','QUOTE_DATE','DTE','C_VOLUME','C_LAST','C_SIZE','C_BID','C_ASK','STRIKE']]

# Change DTE from days to year, and using C_BID and C_ASK to estimate market price (Bid-Ask spread)
traded_options['DTE'] = traded_options['DTE']/365
traded_options = traded_options.rename(columns={'DTE': 'MATURITIES'})
traded_options['MARKET_PRICE'] = (traded_options['C_ASK']+traded_options['C_BID'])/2

# Mapping Nelson Siegel Svensson result function to date, then applying it to the list of maturities of the same date
#rates_df = rates_df.reset_index()
function_mapping = dict(zip(rates_df['Date'], rates_df['curve_fit']))
traded_options = pd.merge(traded_options, rates_df[['Date','curve_fit']], left_on = 'QUOTE_DATE', right_on = 'Date', how='outer')

def apply_function(row):
    func = function_mapping.get(row['Date'])
    if func:
        return func(row['MATURITIES'])
    else:
        return np.nan 

traded_options['RISK_NEUTRAL_RATE'] = traded_options.apply(apply_function, axis=1)

# Interpolation to fill missing risk neutral rates
traded_options['RISK_NEUTRAL_RATE'] = traded_options['RISK_NEUTRAL_RATE'].interpolate(method = 'nearest')

# Dropping unused columns and nulls
traded_options = traded_options.drop(columns=['Date', 'curve_fit'])
traded_options = traded_options.dropna()

traded_options


,UNDERLYING_LAST,QUOTE_DATE,MATURITIES,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,MARKET_PRICE,RISK_NEUTRAL_RATE
0,447.79,2023-10-02,0.010959,1217.0,27.50,39 x 14,28.35,28.85,420.0,28.600,0.057145
1,447.79,2023-10-02,0.010959,1231.0,22.36,21 x 17,23.90,24.25,425.0,24.075,0.057145
2,447.79,2023-10-02,0.010959,1677.0,19.00,25 x 18,19.55,19.80,430.0,19.675,0.057145
3,447.79,2023-10-02,0.010959,963.0,15.50,16 x 34,15.50,15.80,435.0,15.650,0.057145
4,447.79,2023-10-02,0.010959,2202.0,12.05,23 x 6,12.00,12.15,440.0,12.075,0.057145
...,...,...,...,...,...,...,...,...,...,...,...
702,407.80,2023-10-31,0.027507,760.0,13.90,11 x 25,13.85,14.00,405.0,13.925,0.056947
703,407.80,2023-10-31,0.027507,686.0,3.79,152 x 19,3.75,3.85,430.0,3.800,0.056947
704,407.80,2023-10-31,0.046685,631.0,2.80,517 x 218,3.00,3.15,447.5,3.075,0.056849
705,407.80,2023-10-31,0.046685,671.0,0.38,41 x 34,0.40,0.42,485.0,0.410,0.056849


# Calibration of data using a least squared error fit
To find the set of parameters that minimizes the square error:
sqErr(v0, kappa, theta, sigma, rho, lambd) = sum of (Market call price - heston model call price) squared 

Using Scipy (optimization) for minimizing sqErr:
- Problem 1: selecting suitable weight terms (for better calibration results: fitting to more important/reliable data) and penalty  (to avoid overfitting)

- Problem 2: given the problem (minimizing square err), what is the most suitable optimization method to use
    - non linear problem with non linear constraints -> 

- Problem 3: selecting suitable initial parameters and bounds (for convergence speed, avoiding local minima, good quality result)

The result for the parameters should be at a reasonable value, and have a low error when comparing model and real price



In [158]:
# Variables from options data
S0, K, r, tau, price = traded_options[['UNDERLYING_LAST', 'STRIKE', 'RISK_NEUTRAL_RATE', 'MATURITIES','MARKET_PRICE']].astype(float).to_numpy().T

# Parameters （Setting initial guess and upper lower bound)
params = {
    "v0": {"x0": 0.04, "lbub": [1e-3, 0.3]},
    "kappa": {"x0": 3, "lbub": [1e-2, 5]},
    "theta": {"x0": 0.04, "lbub": [1e-3, 0.2]},
    "sigma": {"x0": 0.6, "lbub": [1e-2, 2]},
    "rho": {"x0": -0.8, "lbub": [-1, 1]},
    "lambd": {"x0": 0.8, "lbub": [-1, 1]}
}

inverseMaturitiesSum = (1/tau).sum()

x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

In [151]:
def sqErr (x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    
    # weight factor using inverse weighted average for maturities
    err = np.sum(1/tau/inverseMaturitiesSum*(price - heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd))**2)
    # Penalty term: distance to initial parameter vector
    pen = np.sum([(param-initial)**2 for param, initial in zip(x,x0)])
    
    return err + pen


In [159]:
result = minimize(sqErr, x0, tol = 1e-3, method='SLSQP', options={'maxiter': 1e4 }, bounds=bnds)
result

/var/folders/ts/wphp397x1197fdj1s_5wjk040000gn/T/ipykernel_1815/3704850999.py:19: RuntimeWarning:

overflow encountered in exp

/var/folders/ts/wphp397x1197fdj1s_5wjk040000gn/T/ipykernel_1815/3704850999.py:21: RuntimeWarning:

invalid value encountered in multiply



 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.4928871329601567
       x: [ 1.677e-01  3.005e+00  2.000e-01  5.425e-01 -7.604e-01
            7.898e-01]
     nit: 10
     jac: [-1.265e-01  1.195e-03 -1.386e-01 -2.147e-03 -3.098e-03
            9.169e-04]
    nfev: 74
    njev: 10

In [160]:
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

(np.float64(0.16770359532356446),
 np.float64(3.005235843028312),
 np.float64(0.2),
 np.float64(0.5425074151237366),
 np.float64(-0.7603851177654114),
 np.float64(0.789836262900246))

In [162]:
heston_prices = heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd)
traded_options['HESTON_PRICE'] = heston_prices
traded_options

,UNDERLYING_LAST,QUOTE_DATE,MATURITIES,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,MARKET_PRICE,RISK_NEUTRAL_RATE,HESTON_PRICE
0,447.79,2023-10-02,0.010959,1217.0,27.50,39 x 14,28.35,28.85,420.0,28.600,0.057145,28.799750
1,447.79,2023-10-02,0.010959,1231.0,22.36,21 x 17,23.90,24.25,425.0,24.075,0.057145,24.253579
2,447.79,2023-10-02,0.010959,1677.0,19.00,25 x 18,19.55,19.80,430.0,19.675,0.057145,19.953983
3,447.79,2023-10-02,0.010959,963.0,15.50,16 x 34,15.50,15.80,435.0,15.650,0.057145,15.980400
4,447.79,2023-10-02,0.010959,2202.0,12.05,23 x 6,12.00,12.15,440.0,12.075,0.057145,12.410147
...,...,...,...,...,...,...,...,...,...,...,...,...
702,407.80,2023-10-31,0.027507,760.0,13.90,11 x 25,13.85,14.00,405.0,13.925,0.056947,12.871839
703,407.80,2023-10-31,0.027507,686.0,3.79,152 x 19,3.75,3.85,430.0,3.800,0.056947,3.224432
704,407.80,2023-10-31,0.046685,631.0,2.80,517 x 218,3.00,3.15,447.5,3.075,0.056849,2.162298
705,407.80,2023-10-31,0.046685,671.0,0.38,41 x 34,0.40,0.42,485.0,0.410,0.056849,-0.287899


In [171]:
traded_options[(traded_options['MARKET_PRICE'] - traded_options ['HESTON_PRICE'])<-2]

,UNDERLYING_LAST,QUOTE_DATE,MATURITIES,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,MARKET_PRICE,RISK_NEUTRAL_RATE,HESTON_PRICE
132,446.70,2023-10-05,1.287781,743.0,134.90,7 x 6,133.15,134.15,390.0,133.650,0.051992,136.040536
153,457.62,2023-10-06,0.038356,631.0,20.65,79 x 6,20.15,20.80,445.0,20.475,0.057273,22.584431
264,468.09,2023-10-11,0.024658,555.0,10.60,152 x 1,10.55,10.70,467.5,10.625,0.057348,12.638117


In [155]:
# Visualization
fig = go.Figure(data = [go.Mesh3d(z=traded_options['MARKET_PRICE'], x=traded_options['MATURITIES'], y=traded_options['STRIKE'], colorscale='Viridis', opacity=0.55)])

# Add scatter plot
fig.add_trace(go.Scatter3d(z=traded_options['HESTON_PRICE'], x=traded_options['MATURITIES'], y=traded_options['STRIKE'], mode='markers', marker=dict(size=3, color='red', opacity=0.8)))

# Update layout
fig.update_layout(title='3D Surface Plot with Scatter Points', scene=dict(xaxis_title='Maturities', yaxis_title='Strike', zaxis_title='Price'))

# Show the figure
fig.show()

The results for parameters from optimization seems to be at a reasonable range, and the heston model price prediction overall matches decently well with the market price. It may be normal that the model doesn't fit certain points at the lower and upper end of strike prices due to low liquidity of these options, and the risk of overfitting the model. 

Still, there are many features that can be improved/ added in the future:
- Apply Jump diffusion to Heston model implementation for better calibration accuracy and speed
- Time dependant parameters if considering maturities with large time intervals
- C++ for speed
- Better calibration method for the dataset


# Model Validation using Monte Carlo methods

In [ ]:
def heston_monte_carlo_prices(S0, v0, tau, r, K, sigma, rho, kappa, theta, paths, steps):
    option_values = []
    # Loop through each option (perform a monte carlo simulation for each option)
    for S, r, K, tau in zip(S0, r, K, tau):
        
        dt = tau / steps
        # Arrays to store the final asset prices
        final_prices = np.zeros(paths)

        # Loop for simulating paths 
        for i in range(paths):
            S_t = S
            v_t = v0
            for j in range(steps):
                # Generate correlated wiener processes for price and volatility
                Z1 = np.random.normal(0, 1)
                Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.normal(0, 1)
                
                # Applying heston discrete SDE 

                S_t = S_t*(np.exp((r- 0.5*v_t) * dt + np.sqrt(v_t) * Z1 * np.sqrt(dt)))
                v_t = np.abs(v_t + kappa * (theta - v_t) * dt + sigma * np.sqrt(v_t) * Z2 * np.sqrt(dt))
                
            final_prices[i] = S_t
            
        # Calculating option value by discounting expected payoff to present
        option_value = np.exp(-r * tau) * np.mean(np.maximum(final_prices - K, 0))
        option_values.append(option_value)
            
    return option_values

In [ ]:
option_values = heston_monte_carlo_prices(S0, v0, tau, r, K, sigma, rho, kappa, theta, 100, 252)

TypeError: 'int' object is not iterable

In [ ]:
traded_options['MONTE_CARLO_PRICE'] = option_values
traded_options

,UNDERLYING_LAST,MATURITIES,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,MARKET_PRICE,RISK_NEUTRAL_RATE,HESTON_PRICE,MONTE_CARLO_PRICE
0,447.79,0.010959,1.0,373.60,6 x 7,377.35,377.75,70.0,377.550,0.057145,375.855150,376.983480
30,447.79,0.010959,1.0,138.60,12 x 18,142.10,144.00,305.0,143.050,0.057145,142.988137,149.285260
36,447.79,0.010959,1.0,120.22,7 x 7,122.50,123.35,325.0,122.925,0.057145,123.043755,124.726931
48,447.79,0.010959,1.0,90.13,7 x 10,92.25,93.50,355.0,92.875,0.057145,93.110014,90.043482
50,447.79,0.010959,6.0,88.70,1 x 16,87.55,88.85,360.0,88.200,0.057145,88.119245,87.539799
...,...,...,...,...,...,...,...,...,...,...,...,...
2634,447.79,2.293260,30.0,141.40,40 x 46,140.00,142.65,450.0,141.325,0.050573,155.194116,72.607199
2644,447.79,2.293260,32.0,123.00,31 x 7,121.80,124.00,500.0,122.900,0.050573,133.674399,91.357599
2654,447.79,2.293260,1.0,106.65,1 x 7,105.75,108.05,550.0,106.900,0.050573,115.287892,62.593197
2675,447.79,2.293260,2.0,69.79,26 x 34,69.70,71.10,700.0,70.400,0.050573,75.360321,25.426304


Testing accuracy of calibrated parameters (using data from October) with November data 